In [ ]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
import pyvista as pv
import pvxarray

from pv_glacier_source import PyVistaGlacierSource

pv.set_jupyter_backend("trame")

In [ ]:
ds_glacier = xr.open_dataset("data/topo_and_distributed_glacier_evolution.nc")

In [ ]:
ds_glacier = ds_glacier.isel(x=slice(150, 350), y=slice(150, 350)).load()

ds_glacier

In [ ]:
from skimage.exposure.exposure import rescale_intensity

def _rescale_intensity_pc(arr):
    p02_98 = np.quantile(arr, [0.15, 0.85])
    rescaled = rescale_intensity(arr, in_range=tuple(p02_98), out_range=(0, 255))
    return rescaled.astype(np.uint8)


def xr_rescale_intensity(da):
    return xr.apply_ufunc(
        _rescale_intensity_pc,
        da,
        input_core_dims=[["x", "y"]],
        output_core_dims=[["x", "y"]],
        vectorize=True
    )

In [ ]:
import pystac_client
import pyproj
from xrspatial.multispectral import true_color

x_min = ds_glacier.x.min()
x_max = ds_glacier.x.max()
y_min = ds_glacier.y.min()
y_max = ds_glacier.y.max()

utm_proj = pyproj.Proj(ds_glacier.attrs["pyproj_srs"])
lon_min, lat_min = utm_proj(x_min, y_min, inverse=True)
lon_max, lat_max = utm_proj(x_max, y_max, inverse=True)

catalog = pystac_client.Client.open(
    "https://earth-search.aws.element84.com/v0"
)
catalog.add_conforms_to("ITEM_SEARCH")
catalog.add_conforms_to("QUERY")

search = catalog.search(
    collections=["sentinel-s2-l2a-cogs"],
    bbox=[lon_min, lat_min, lon_max, lat_max],
    query={
        "eo:cloud_cover": {"lt": 10},
        "sentinel:valid_cloud_cover": {"eq": True},
    },
    datetime="2020-07-01/2020-10-01",
    max_items=30,
)

da_img_raw = (
    xr.open_dataset(search, engine="stac")
    .sel(x=slice(x_min, x_max), y=slice(y_max, y_min))
    .get(["B04", "B03", "B02"])
    .to_array(dim="band")
)

da_img_raw

In [ ]:
da_img_raw = da_img_raw.compute()

In [ ]:
da_img_rgb = (
    da_img_raw
    .median(dim="time")
    .pipe(lambda da: true_color(*da))
)

In [ ]:
da_img_rgb.plot.imshow(x="x", y="y", rgb="band", aspect=1, size=6);

In [ ]:
img_texture = pv.Texture(da_img_rgb.transpose("y", "x", "band").values)

In [ ]:
topo_bedrock_mesh = ds_glacier.topo_bed_rock.pyvista.mesh(x="x", y="y")

da_thickness = ds_glacier["simulation_distributed_thickness_BCC-CSM2-MR_ssp126"]
da_glacier_surf = ds_glacier.topo_bed_rock + da_thickness

In [ ]:
from pyvista import examples

cubemap = examples.download_sky_box_cube_map()

In [ ]:
warped = topo_bedrock_mesh.warp_by_scalar()
warped.texture_map_to_plane(use_bounds=True, inplace=True);

In [ ]:
glacier_algo = PyVistaGlacierSource(da_glacier_surf)

In [ ]:
pl = pv.Plotter(lighting="none")

pl.add_mesh(warped, texture=img_texture)
pl.add_mesh(glacier_algo, color="pink")

pl.add_text(
    f"year: {glacier_algo.time}",
    position="upper_right",
    font_size=16,
    name="current_year",
)

light = pv.Light(
    position=(0, 1, 1),
    light_type="scene light",
)
pl.add_light(light)

pl.set_background("white", top="lightblue")
#pl.set_environment_texture(cubemap)
#pl.add_actor(cubemap.to_skybox())
#pl.enable_eye_dome_lighting()

In [ ]:
max_time = da_glacier_surf.time.size

play = widgets.Play(
    value=0,
    min=0,
    max=max_time - 1,
    step=1,
    interval=200,
    description="Press play",
    disabled=False,
)
slider = widgets.IntSlider(min=0, max=max_time - 1, step=1)
widgets.jslink((play, "value"), (slider, "value"))

def update_glacier(change):
    glacier_algo.time_step = change["new"]
    glacier_algo.update()
    pl.add_text(
        f"year: {glacier_algo.time}",
        position="upper_right",
        font_size=16,
        name="current_year",
    )
    pl.update()

slider.observe(update_glacier, names="value")

output = widgets.Output()

with output:
    pl.show()

widgets.VBox([widgets.HBox([play, slider]), output])